# HW2 Problem 3

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [60]:
def get_distance_angle(A, B):
    """ Returns distance and angle of the line between two points.
        Input: 
            A - numpy array of [lat, lon] in degree
            B - numpy array of [lat, lon] in degree
        Return:
            L_AB - np.array([L_NS (km), L_EW (km)])
            A_AB - Angle of line AB measured from the E-W line, positive anticlockwise
    """
    R = 6371.0 # km.
    dist_factor = np.pi*R/180.0
    phiAB = 0.5*(A[0]+B[0])*np.pi/180.0
    
    L_NS = (B[1]-A[1])*dist_factor   #((B_lat)-(A_lat))*((np.pi*6378)/180)
    L_EW = (B[0]-A[0])*dist_factor*phiAB
    L_AB = np.array([L_NS, L_EW])
    
    # Note: ||v|| corresponds to np.linalg.norm(v), 
    # where v is a vector as a numpy array.
    Dist_AB = np.linalg.norm(L_AB)
    print("Distance: {0:8.4f} km".format(Dist_AB))

    A_AB = np.arctan(L_NS/L_EW)
    print("Angle: {0:8.4f} deg ".format(A_AB*180.0/np.pi))
    
    return L_AB, A_AB

def get_vel_mag_dir(V):
    """ Computes and prints magnitude and direction of a given velocity vector.
        Input:
            V - velocity vector, np.array([V_north, V_east])
        Output:
            prints magnitude (||V||) in mm/yr and direction in degrees (atan(V_E/V_N)) relative to North are returned.
    """
    # Note: ||v|| corresponds to np.linalg.norm(v), 
    # where v is a vector as a numpy array.

    vel_mag = np.linalg.norm(V)*1e6 # To be completed. Be careful about unit. 
    vel_dir = (np.arctan(V[1]/V[0]))*180/np.pi # To be completed. Be careful about unit.
    # magnitude in mm/yr and direction relative to North.
    print("mag: {0:8.4f} mm/yr, direction: {1:8.4f} deg".format(vel_mag, vel_dir))
    
def get_Ldot_edot(L_AB, A_AB, VA, VB):
    """ Computes the time rate of change of the distance between A and B; and strain rate
        along the line connecting A and B.
        Input:
            L_AB - Distance vector returned from get_distance_angle() function
            A_AB - Angle returned from get_distance_angle() function
            VA   - Velocity vector of monument A: [VA_north, VA_east]
            VB   - Velocity vector of monument B: [VB_north, VB_east]
        Return:
            Ldot_AB - Rate of change of distance between A and B
            edot_AB - Strain rate along the line AB, Ldot_AB/L_AB.
    """
    Ldot_AB = (VB[0] - VA[0])*np.sin(A_AB) + (VB[1] - VA[1])*np.cos(A_AB)  # To be completed.
    edot_AB = -(Ldot_AB*1e-6/np.linalg.norm(L_AB)) # To be completed.
    return Ldot_AB, edot_AB

## 1) Problem 2-34 of T&S

In [62]:
# Data for Problem 2.34
A = np.array([38.3137, -123.0689]) # lat, lon degrees
B = np.array([38.4442, -122.1953])
C = np.array([38.7778, -122.5758])

VA = np.array([33.0e-6, -24.0e-6]) # VN, VE km/yr
VB = np.array([10.0e-6, -13.0e-6])
VC = np.array([16.0e-6, -14.0e-6])

### (i) line lengths and directions

In [75]:
L_AB, A_AB = get_distance_angle(A,B)
L_AC, A_AC = get_distance_angle(A,C)
L_BC, A_BC = get_distance_angle(B,C)
# Based on the geometry, A_BC_geom is defined to be between 0 and 90 deg.
A_BC_geom = np.pi - A_BC
#print((A_BC_geom*180.0/np.pi))
print("A_BC_geom = {0:8.4f} deg".format(A_BC_geom*180.0/np.pi))

Distance:  97.6250 km
Angle:  84.2859 deg 
Distance:  64.8974 km
Angle:  57.6586 deg 
Distance:  49.1426 km
Angle: -59.4243 deg 
A_BC_geom = 239.4243 deg


### (ii) velocity vectors: [N, E]. Units: km/yr.

In [64]:
get_vel_mag_dir(VA)
get_vel_mag_dir(VB)
get_vel_mag_dir(VC) 

mag:  40.8044 mm/yr, direction: -36.0274 deg
mag:  16.4012 mm/yr, direction: -52.4314 deg
mag:  21.2603 mm/yr, direction: -41.1859 deg


### (iii) Length change rates (km/yr) and strain rates (/yr)

In [81]:
# Ldot_AB = (VB-VA)[0]*cos(79.2deg) + (VB-VA)[1]*cos(10.8deg).
Ldot_AB, edot_AB = get_Ldot_edot( L_AB, A_AB, VA, VB )

# Ldot_AC = (VC-VA)[0]*cos(39.7deg) + (VC-VA)[1]*cos(50.3deg).
Ldot_AC ,edot_AC = get_Ldot_edot( L_AC, A_AC, VA, VC )

# Ldot_BC = (VC-VB)[0]*cos(41.7deg) + (VC-VB)[1]*cos(48.3deg).
Ldot_BC, edot_BC = get_Ldot_edot( L_BC, A_BC, VB, VC )

# print Ldot's and edot's. Units are km/yr and /yr, respectively.
print("AB: Ldot={0:8.4e} km/yr, edot={1:8.4e} /yr".format(Ldot_AB, edot_AB))
print("AC: Ldot={0:8.4e} km/yr, edot={1:8.4e} /yr".format(Ldot_AC, edot_AC))
print("BC: Ldot={0:8.4e} km/yr, edot={1:8.4e} /yr".format(Ldot_BC, edot_BC))

AB: Ldot=-2.1791e-05 km/yr, edot=2.2321e-13 /yr
AC: Ldot=-9.0133e-06 km/yr, edot=1.3888e-13 /yr
BC: Ldot=-5.6744e-06 km/yr, edot=1.1547e-13 /yr


### (iv) $\theta_{1}$ ($^{\circ}$), $\theta_{2}$ ($^{\circ}$), $\dot{\varepsilon}_{zz}$ (/yr), and $\dot{\varepsilon}_{xz}$ (/yr)

In [96]:
theta1 = (A_AC - A_AB) # To be completed
theta2 = ((np.pi/180 * 180) - A_AB - A_BC) # To be completed

cot1 = 1.0/np.tan(theta1)
cot2 = 1.0/np.tan(theta2)
sec1 = 1.0/np.cos(theta1)
sec2 = 1.0/np.cos(theta2)
csc1 = 1.0/np.sin(theta1)
csc2 = 1.0/np.sin(theta2)

edot_zz = (edot_AB*(cot1 - cot2) - edot_AC*sec1*csc1 + edot_BC*sec2*csc2)/(np.tan(theta2) - np.tan(theta1))
edot_xz = (edot_AB*(cot1**2 - cot2**2) - edot_AC*csc1**2 + edot_BC*csc2**2)/(2*(cot2 - cot1))
print("theta1 = {0:8.4f} deg, theta2 = {1:8.4f} deg".format(theta1*180.0/np.pi, theta2*180.0/np.pi))
print("edot_zz = {0:8.4e} /yr, edot_xz = {1:8.4e} /yr".format(edot_zz, edot_xz))

theta1 = -26.6273 deg, theta2 = 155.1384 deg
edot_zz = 2.1178e-12 /yr, edot_xz = 5.8016e-13 /yr


### 2) Problem 2-35 

In [97]:
# Data for Problem 2.35
A = np.array([37.1924, -122.3669]) # lat, lon degrees
B = np.array([36.8063, -120.7431])
C = np.array([37.7553, -121.4640])

VA = np.array([36.0e-6, -29.0e-6]) # VN, VE km/yr
VB = np.array([9.0e-6, -10.0e-6])
VC = np.array([8.0e-6, -12.0e-6])

### (i) line lengths and directions

In [98]:
# Repeat the procedure for Problem 2-34
L_AB, A_AB = get_distance_angle(A,B)
L_AC, A_AC = get_distance_angle(A,C)
L_BC, A_BC = get_distance_angle(B,C)
# Based on the geometry, A_BC_geom is defined to be between 0 and 90 deg.
A_BC_geom = np.pi - A_BC
#print((A_BC_geom*180.0/np.pi))
print("A_BC_geom = {0:8.4f} deg".format(A_BC_geom*180.0/np.pi))

Distance: 182.6744 km
Angle: -81.2706 deg 
Distance: 108.4233 km
Angle:  67.8167 deg 
Distance: 105.5467 km
Angle: -49.4182 deg 
A_BC_geom = 229.4182 deg


### (ii) velocity vectors: [N, E]. Units: km/yr.

In [99]:
# Repeat the procedure for Problem 2-34
get_vel_mag_dir(VA)
get_vel_mag_dir(VB)
get_vel_mag_dir(VC)

mag:  46.2277 mm/yr, direction: -38.8534 deg
mag:  13.4536 mm/yr, direction: -48.0128 deg
mag:  14.4222 mm/yr, direction: -56.3099 deg


### (iii) Length change rates (km/yr) and strain rates (/yr)

In [101]:
# Repeat the procedure for Problem 2-34
# Ldot_AB = (VB-VA)[0]*cos(79.2deg) + (VB-VA)[1]*cos(10.8deg).
Ldot_AB, edot_AB = get_Ldot_edot( L_AB, A_AB, VA, VB )

# Ldot_AC = (VC-VA)[0]*cos(39.7deg) + (VC-VA)[1]*cos(50.3deg).
Ldot_AC ,edot_AC = get_Ldot_edot( L_AC, A_AC, VA, VC )

# Ldot_BC = (VC-VB)[0]*cos(41.7deg) + (VC-VB)[1]*cos(48.3deg).
Ldot_BC, edot_BC = get_Ldot_edot( L_BC, A_BC, VB, VC )

# print Ldot's and edot's. Units are km/yr and /yr, respectively.
print("AB: Ldot = {0:8.4e} km/yr, edot = {1:8.4e} /yr".format(Ldot_AB, edot_AB))
print("AC: Ldot = {0:8.4e} km/yr, edot = {1:8.4e} /yr".format(Ldot_AC, edot_AC))
print("BC: Ldot = {0:8.4e} km/yr, edot = {1:8.4e} /yr".format(Ldot_BC, edot_BC))

AB: Ldot = 2.9571e-05 km/yr, edot = -1.6188e-13 /yr
AC: Ldot = -1.9509e-05 km/yr, edot = 1.7993e-13 /yr
BC: Ldot = -5.4159e-07 km/yr, edot = 5.1313e-15 /yr


### (iv) $\theta_{1}$ ($^{\circ}$), $\theta_{2}$ ($^{\circ}$), $\dot{\varepsilon}_{zz}$ (/yr), and $\dot{\varepsilon}_{xz}$ (/yr)

In [102]:
# Repeat the procedure for Problem 2-34
theta1 = (A_AC - A_AB) # To be completed
theta2 = ((np.pi/180 * 180) - A_AB - A_BC) # To be completed

cot1 = 1.0/np.tan(theta1)
cot2 = 1.0/np.tan(theta2)
sec1 = 1.0/np.cos(theta1)
sec2 = 1.0/np.cos(theta2)
csc1 = 1.0/np.sin(theta1)
csc2 = 1.0/np.sin(theta2)

edot_zz = (edot_AB*(cot1 - cot2) - edot_AC*sec1*csc1 + edot_BC*sec2*csc2)/(np.tan(theta2) - np.tan(theta1))
edot_xz = (edot_AB*(cot1**2 - cot2**2) - edot_AC*csc1**2 + edot_BC*csc2**2)/(2*(cot2 - cot1))
print("theta1 = {0:8.4f} deg, theta2 = {1:8.4f} deg".format(theta1*180.0/np.pi, theta2*180.0/np.pi))
print("edot_zz = {0:8.4e} /yr, edot_xz = {1:8.4e} /yr".format(edot_zz, edot_xz))

theta1 = 149.0873 deg, theta2 = 310.6889 deg
edot_zz = -9.3751e-13 /yr, edot_xz = -6.1997e-13 /yr
